## 算法介绍

  inception v3：
  
  - 将大尺度卷积分解为多个小尺度卷积减少计算量。如将一个5x5卷积替换为两个3x3卷积，在减少网络参数的同时提升了网络深度  
  - 使用非对称卷积。如将一个nxn卷积替换为两个1xn，nx1卷积，对于3x3卷积，在相同卷积核数量下，这样操作可以节省33%的计算量  
  - 使用并行结构来优化Pooling。 即使用两个并行的支路，一路采用1x1卷积， 一路采用Pooling，最后再在特征维度拼合到一起，这种方法既有很好的效果，又没有增大计算量  
  - 使用了BN操作
  - 输入从(224, 224) 提升至 (299, 299)
  - averagepooling 和 Maxpooling 搭配使用
  - 采用RMSProp寻优方法
  - 采用dropout用于防止过拟合
  - 使用BN-auxiliary，使得网络整体效果提升0.4%
  - 整体架构使用了以下5种inception module结构(从上而下依次为Inception-A，Inception-B，Inception-C，Inception-D，Inception-E)：
  
  ![fig1](fig1.png)
  
  ![fig2](fig2.png)
  
  ![fig3](fig3.png)
  
  ![fig4](fig4.png)
  
  ![fig5](fig5.png)
  
  下面我们基于上述描述的结构对inception v3 进行编码实现。

## 算法实现

In [1]:
import keras

#conv_layer
def conv_layer(x, filters, kernel_size, padding = 'same', strides = (1, 1), name = None):
    '''
    conv + bn + relu
    :param x: tensor, tensor of previous layer
    :param filters: integer, number of kernel
    :param kernel_size: tumple, kernel's (row, col)
    :param padding: 'same' or 'valid'
    :param strides: tumple, stride of kernel
    :param name: layer's name
    
    returns:
    output tensor
    '''
    x = keras.layers.Conv2D(filters, kernel_size, strides = strides, padding = padding, use_bias = False, name = name + '_conv')(x)
    x = keras.layers.BatchNormalization(axis = 3, name = name + '_bn')(x)
    x = keras.layers.Activation('relu', name = name + '_relu')(x)
    
    return x

/usr/local/SkyCompute/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
#inception-A
def inceptionA(x, f_branch1_1x1,
              f_branch2_5x5,
              f_branch3_3x3,
              f_branch4_reducepool,
              f_branch2_reduce5,
              f_branch3_reduce3,
              name=None):
    '''
    inception A
    :param x: input tensor
    :param f_branch1_1x1: integer, number of kernel of 1*1
    :param f_branch2_5x5: integer, number of kernel of 5*5
    :param f_branch4_3x3: list, number of kernel of double 3*3
    :param branch2_reducepool: integer, number of reduce channel 1*1 of pool
    :param f_branch2_reduce5: integer, number of reduce channel 1*1 of 5*5 conv
    :param f_branch3_reduce3: integer, number of reduce channel 1*1 of double 3*3 conv
    :param name: string, layer's name
    
    returns:
    output tensor
    '''
    #conv 1*1
    branch1_1x1 = conv_layer(x = x, filters = f_branch1_1x1, kernel_size = (1, 1), name = name + '_branch1_1x1')
    
    #conv 5*5
    branch2_5x5 = conv_layer(x = x, filters = f_branch2_reduce5, kernel_size = (1, 1), name = name + '_branch2_1x1')
    branch2_5x5 = conv_layer(x = branch2_5x5, filters = f_branch2_5x5, kernel_size = (5, 5), name = name + '_branch2_5x5')
    
    #double conv 3*3
    branch3_3x3 = conv_layer(x = x, filters = f_branch3_reduce3, kernel_size = (1, 1), name = name + '_branch3_1x1')
    branch3_3x3 = conv_layer(x = branch3_3x3, filters = f_branch3_3x3[0], kernel_size = (3, 3), name = name + '_branch3_3x3_1')
    branch3_3x3 = conv_layer(x = branch3_3x3, filters = f_branch3_3x3[1], kernel_size = (3, 3), name = name + '_branch3_3x3_2')
    
    #pool
    branch4_pool = keras.layers.AveragePooling2D(pool_size = (3, 3), strides = (1, 1), padding='same', name = name + '_branch4_pool')(x)
    branch4_pool = conv_layer(x = branch4_pool, filters = f_branch4_reducepool, kernel_size = (1, 1), name = name + '_branch4_1x1')
    
    x = keras.layers.concatenate([branch1_1x1, branch2_5x5, branch3_3x3, branch4_pool], axis = 3, name = name)
    return x

In [3]:
#inception-B
def inceptionB(x, f_branch1_3x3,
               f_branch2_3x3,
               f_branch2_reduce3,
              name = None):
    '''
    inception B
    :param x: input tensor
    :param f_branch1_3x3: integer, number of kernel of 3*3
    :param f_branch2_3x3: list, number of kernel of double 3*3
    :param f_branch2_reduce3: integer, number of reduce channel 1*1 of double 3*3 conv
    
    returns:
    output tensor
    '''
    #conv_3
    branch1_3x3 = conv_layer(x = x, filters = f_branch1_3x3, kernel_size = (3, 3), strides=(2, 2), padding = 'valid', name = name + '_branch1_3x3')
    
    #double_conv33
    branch2_3x3 = conv_layer(x = x, filters = f_branch2_reduce3, kernel_size = (1, 1), name = name + '_branch2_1x1')
    branch2_3x3 = conv_layer(x = branch2_3x3, filters = f_branch2_3x3[0], kernel_size = (3, 3), name = name + '_branch2_3x3_1')
    branch2_3x3 = conv_layer(x = branch2_3x3, filters = f_branch2_3x3[1], kernel_size = (3, 3), strides = (2, 2), padding = 'valid', name = name + '_branch2_3x3_2')
    
    #pool
    branch3_pool = keras.layers.MaxPooling2D(pool_size = (3, 3), strides = (2, 2), name = name + '_branch3_pool')(x)
    
    x = keras.layers.concatenate([branch1_3x3, branch2_3x3, branch3_pool], axis=3, name = name)
    return x

In [4]:
#inception-C
def inceptionC(x, f_branch1_1x1,
               f_branch2_7x7,
               f_branch3_7x7,
               f_branch2_reduce7,
               f_branch3_reduce7,
               f_branch4_reducepool,
               name=None):
    '''
    inception C
    :param x: input tensor
    :param f_branch1_1x1: integer, number of kernels of conv 1*1
    :param f_branch2_7x7: list, number of kernels of conv 1*7 and 7*1
    :param f_branch3_7x7: list, number of kernels of conv 7*1, 1*7, 7*1, 1*7
    :param f_branch2_reduce7: integer, number of kernels of conv 1*1 reducing 1*7, 7*1
    :param f_branch3_reduce7: integer, number of kernels of conv 1*1 reducing 7*1, 1*7, 7*1, 1*7
    :param f_branch4_reducepool: integer, number of kernels of conv 1*1 reducing pooling
    
    returns:
    output tensor
    '''
    #conv1
    branch1_1x1 = conv_layer(x = x, filters = f_branch1_1x1, kernel_size = (1, 1), name = name + '_branch1_1x1')
    
    #conv7
    branch2_7x7 = conv_layer(x = x, filters = f_branch2_reduce7, kernel_size = (1, 1), name = name + '_branch2_1x1')
    branch2_7x7 = conv_layer(x = branch2_7x7, filters = f_branch2_7x7[0], kernel_size = (1, 7), name = name + '_branch2_1x7')
    branch2_7x7 = conv_layer(x = branch2_7x7, filters = f_branch2_7x7[1], kernel_size = (7, 1), name = name + '_branch2_7x1')
    
    #db_conv7
    branch3_7x7 = conv_layer(x = x, filters = f_branch3_reduce7, kernel_size = (1, 1), name = name + '_branch3_1x1')
    branch3_7x7 = conv_layer(x = branch3_7x7, filters = f_branch3_7x7[0], kernel_size = (7, 1), name = name + '_branch3_7x1_1')
    branch3_7x7 = conv_layer(x = branch3_7x7, filters = f_branch3_7x7[1], kernel_size = (1, 7), name = name + '_branch3_1x7_1')
    branch3_7x7 = conv_layer(x = branch3_7x7, filters = f_branch3_7x7[2], kernel_size = (7, 1), name = name + '_branch3_7x1_2')
    branch3_7x7 = conv_layer(x = branch3_7x7, filters = f_branch3_7x7[3], kernel_size = (1, 7), name = name + '_branch3_1x7_2')
    
    #pool
    branch4_pool = keras.layers.AveragePooling2D(pool_size = (3, 3), strides=(1, 1), padding='same', name = name + '_branch4_pool')(x)
    branch4_pool = conv_layer(x = branch4_pool, filters = f_branch4_reducepool, kernel_size = (1, 1), name = name + '_branch4_1x1')
    
    x = keras.layers.concatenate([branch1_1x1, branch2_7x7, branch3_7x7, branch4_pool], axis = 3, name = name)
    return x

In [5]:
#inception-D
def inceptionD(x, f_branch1_3x3,
              f_branch2_7x7,
              f_branch2_3x3,
              f_branch1_reduce3,
              f_branch2_reduce7,
              name = None):
    '''
    inception D
    :param x: input tensor
    :param f_branch1_3x3: integer, number of kernels 3*3 in branch 1
    :param f_branch2_7x7: list, number of kernels 1*7, 7*1 in branch 2
    :param f_branch2_3x3: integer, number of kernels 3*3 in branch 2
    :param f_branch1_reduce3: integer, number of kernels 1*1 reducing channels of 3*3
    :param f_branch2_reduce7: integer, number of kernels 1*1 reducing channels of 1*7, 7*1, 3*3
    
    returns:
    output tensor
    '''
    #branch1 conv3
    branch1_3x3 = conv_layer(x = x, filters = f_branch1_reduce3, kernel_size = (1, 1), name = name + '_branch1_1x1')
    branch1_3x3 = conv_layer(x = branch1_3x3, filters = f_branch1_3x3, kernel_size = (3, 3), strides = (2, 2), padding = 'valid', name = name + '_branch1_3x3')
    
    #branch2 conv7, conv3
    branch2_7x7 = conv_layer(x = x, filters = f_branch2_reduce7, kernel_size = (1, 1), name = name + '_branch2_1x1')
    branch2_7x7 = conv_layer(x = branch2_7x7, filters = f_branch2_7x7[0], kernel_size = (1, 7), name = name + '_branch2_1x7')
    branch2_7x7 = conv_layer(x = branch2_7x7, filters = f_branch2_7x7[1], kernel_size = (7, 1), name = name + '_branch2_7x1')
    branch2_3x3 = conv_layer(x = branch2_7x7, filters = f_branch2_3x3, kernel_size = (3, 3), strides = (2, 2), padding = 'valid', name = name + '_branch2_3x3')
    
    #pool
    branch3_pool = keras.layers.MaxPooling2D(pool_size = (3, 3), strides = (2, 2), name = name + '_branch3_pool')(x)
    
    x = keras.layers.concatenate([branch1_3x3, branch2_3x3, branch3_pool], axis = 3, name = name)
    return x

In [6]:
#inception-E
def inceptionE(x, f_branch1_1x1,
               f_branch2_3x3_sp,               
               f_branch3_3x3,
               f_branch3_3x3_sp,
               f_branch2_reduce3,
               f_branch3_reduce3,
               f_branch4_reducepool,
               name = None):
    '''
    inception E
    :param x: input tensor
    :param f_branch1_1x1: integer, number of kernels 1x1
    :param f_branch2_3x3_sp: list, number of kernels 3x1, 1x3
    :param f_branch3_3x3: integer, number of kernels 3x3
    :param f_branch3_3x3_sp: list, number of kernels 3x1, 1x3
    :param f_branch2_reduce3: integer, number of kernels 1x1 reducing channels of 3x1, 1x3
    :param f_branch3_reduce3: integer, number of kernels 1x1 reducing channels of 3x3
    :param f_branch4_reducepool: integer, number of kernels 1x1 reducing channels of pool
    
    returns:
    output tensor
    '''
    #branch1 conv1
    branch1_1x1 = conv_layer(x = x, filters = f_branch1_1x1, kernel_size = (1, 1), name = name + '_branch1_1x1')
    
    #branch2 conv 3x3
    branch2_1x1 = conv_layer(x = x, filters = f_branch2_reduce3, kernel_size = (3, 3), name = name + '_branch2_1x1')
    branch2_3x3_1 = conv_layer(x = branch2_1x1, filters = f_branch2_3x3_sp[0], kernel_size = (3, 1), name = name + '_branch2_3x3_1')
    branch2_3x3_2 = conv_layer(x = branch2_1x1, filters = f_branch2_3x3_sp[1], kernel_size = (1, 3), name = name + '_branch2_3x3_2')
    branch2_3x3 = keras.layers.concatenate([branch2_3x3_1, branch2_3x3_2], axis = 3, name = name + '_branch2_3x3')
    
    #branch3
    branch3_1x1 = conv_layer(x = x, filters = f_branch3_reduce3, kernel_size = (1, 1), name = name + '_branch3_1x1')
    branch3_3x3 = conv_layer(x = branch3_1x1, filters = f_branch3_3x3, kernel_size = (3, 3), name = name + '_branch3_3x3')
    branch3_3x3_1 = conv_layer(x = branch3_3x3, filters = f_branch3_3x3_sp[0], kernel_size = (3, 1), name = name + '_branch3_3x3_1')
    branch3_3x3_2 = conv_layer(x = branch3_3x3, filters = f_branch3_3x3_sp[1], kernel_size = (1, 3), name = name + '_branch3_3x3_2')
    branch_3x3_f = keras.layers.concatenate([branch3_3x3_1, branch3_3x3_2], axis = 3, name = name + '_branch3_3x3_final')
    
    #branch4
    branch4_pool = keras.layers.AveragePooling2D(pool_size = (3, 3), strides = (1, 1), padding = 'same', name = name + '_branch4_pool')(x)
    branch4_pool = conv_layer(x = branch4_pool, filters = f_branch4_reducepool, kernel_size = (1, 1), name = name + '_branch4_1x1')
    
    x = keras.layers.concatenate([branch1_1x1, branch2_3x3, branch_3x3_f, branch4_pool], axis = 3, name  = name)
    return x

  上述对几个inception module进行了编码实现，其网络的整体架构是由inception module构建而成，网络构架如下表：

  ![fig6](fig6.png)
  
  其中，在inceptionCx4模块后添加了BN-auxiliary，基本结构为：
  
  pool(5x5/3)->conv(1x1)->conv(5x5)->fc
  
  基于结构表，辅助分类器以及整个网络的编码实现如下：

In [13]:
def BN_auxiliary(x, classes):
    '''
    BN auxiliary
    :param x: input tensor(feature map)
    :param classes: number of output class
    
    returns:
    aux_logists
    '''
    x = keras.layers.AveragePooling2D(pool_size = (5, 5), strides = (3, 3), name = 'aux_average_pooling')(x)#5x5x768
    x = conv_layer(x = x, filters = 128, kernel_size = (1, 1), padding = 'same', strides = (1, 1), name = 'aux_conv1')#5x5x128
    x = conv_layer(x = x, filters = 768, kernel_size = (5, 5), padding = 'valid', strides = (1, 1), name = 'aux_conv2')#1x1x768
    x = keras.layers.Flatten(name = 'aux_flatten')(x)#768
    logits = keras.layers.Dense(units = classes, activation='softmax', name = 'aux_classification')(x)
    return logits

In [19]:
def Inceptionv3(input_shape=(299,299,3), classes = 1000, use_dropout = True, dropout_rate = 0.2, use_BN_auxiliary = True):
    '''
    inception v3
    :param input_shape: tumple, network input shape
    :param classes: integer, number of class
    :param use_dropout: bool, if true, dropout operation before Dense, else, only Dense
    :param dropout_rate: float, only valid if use_dropout = True
    :param use_BN_auxiliary: use BN_auxiliary or not
    
    returns:
    keras model
    '''
    x_input = keras.layers.Input(input_shape)
    
    x = conv_layer(x = x_input, filters = 32, kernel_size = (3, 3), strides = (2, 2), padding = 'valid', name = 'conv1_1')
    x = conv_layer(x = x, filters = 32, kernel_size = (3, 3), strides = (1, 1), padding = 'valid', name = 'conv1_2')
    x = conv_layer(x = x, filters = 64, kernel_size = (3, 3), strides = (1, 1), padding = 'same', name = 'conv1_3')
    x = keras.layers.MaxPooling2D(pool_size = (3, 3), strides = (2, 2), padding = 'valid', name = 'maxpool1')(x)
    
    x = conv_layer(x = x, filters = 80, kernel_size = (1, 1), strides = (1, 1), padding = 'valid', name = 'conv2_1')
    x = conv_layer(x = x, filters = 192, kernel_size = (3, 3), strides = (1, 1), padding = 'valid', name = 'conv2_2')
    x = keras.layers.MaxPooling2D(pool_size = (3, 3), strides = (2, 2), padding = 'valid', name = 'maxpool2')(x)
    
    #inception A
    x = inceptionA(x = x, f_branch1_1x1 = 64,
                   f_branch2_5x5 = 64,
                   f_branch3_3x3 = [96, 96],
                   f_branch4_reducepool = 32,
                   f_branch2_reduce5 = 48,
                   f_branch3_reduce3 = 64,
                   name='inceptionA1')
    x = inceptionA(x = x, f_branch1_1x1 = 64,
                   f_branch2_5x5 = 64,
                   f_branch3_3x3 = [96, 96],
                   f_branch4_reducepool = 64,
                   f_branch2_reduce5 = 48,
                   f_branch3_reduce3 = 64,
                   name='inceptionA2')
    x = inceptionA(x = x, f_branch1_1x1 = 64,
                   f_branch2_5x5 = 64,
                   f_branch3_3x3 = [96, 96],
                   f_branch4_reducepool = 64,
                   f_branch2_reduce5 = 48,
                   f_branch3_reduce3 = 64,
                   name='inceptionA3')
    
    #inception B
    x = inceptionB(x = x, f_branch1_3x3 = 384,
                   f_branch2_3x3 = [96, 96],
                   f_branch2_reduce3 = 64,
                   name = 'inceptionB')
    
    #inception C
    x = inceptionC(x = x, f_branch1_1x1 = 192,
               f_branch2_7x7 = [128, 192],
               f_branch3_7x7 = [128, 128, 128, 192],
               f_branch2_reduce7 = 128,
               f_branch3_reduce7 = 128,
               f_branch4_reducepool = 192,
               name='inceptionC1')
    x = inceptionC(x = x, f_branch1_1x1 = 192,
               f_branch2_7x7 = [160, 192],
               f_branch3_7x7 = [160, 160, 160, 192],
               f_branch2_reduce7 = 160,
               f_branch3_reduce7 = 160,
               f_branch4_reducepool = 192,
               name='inceptionC2')
    x = inceptionC(x = x, f_branch1_1x1 = 192,
               f_branch2_7x7 = [160, 192],
               f_branch3_7x7 = [160, 160, 160, 192],
               f_branch2_reduce7 = 160,
               f_branch3_reduce7 = 160,
               f_branch4_reducepool = 192,
               name='inceptionC3')
    x = inceptionC(x = x, f_branch1_1x1 = 192,
               f_branch2_7x7 = [192, 192],
               f_branch3_7x7 = [192, 192, 192, 192],
               f_branch2_reduce7 = 192,
               f_branch3_reduce7 = 192,
               f_branch4_reducepool = 192,
               name='inceptionC4')
    
    #add BN_auxiliary
    if use_BN_auxiliary:
        logits = BN_auxiliary(x = x, classes = classes)
    
    #inception D
    x = inceptionD(x, f_branch1_3x3 = 320,
              f_branch2_7x7 = [192, 192],
              f_branch2_3x3 = 192,
              f_branch1_reduce3 = 192,
              f_branch2_reduce7 = 192,
              name = 'inceptionD')
    
    #inception E
    x = inceptionE(x = x, f_branch1_1x1 = 320,
               f_branch2_3x3_sp = [384, 384],               
               f_branch3_3x3 = 384,
               f_branch3_3x3_sp = [384, 384],
               f_branch2_reduce3 = 384,
               f_branch3_reduce3 = 448,
               f_branch4_reducepool = 192,
               name = 'inceptionE1')
    x = inceptionE(x = x, f_branch1_1x1 = 320,
               f_branch2_3x3_sp = [384, 384],               
               f_branch3_3x3 = 384,
               f_branch3_3x3_sp = [384, 384],
               f_branch2_reduce3 = 384,
               f_branch3_reduce3 = 448,
               f_branch4_reducepool = 192,
               name = 'inceptionE2')
    
    #global_avg_pool
    x = keras.layers.GlobalAveragePooling2D(name = 'global_avg_pooling')(x)
    #dropout
    if use_dropout:
        x = keras.layers.Dropout(dropout_rate, name = 'dropout')(x)#概率可修改为其他参数
    #fc
    x = keras.layers.Dense(units = classes, activation='softmax', name = 'classification')(x)
    
    model = keras.models.Model(inputs = x_input, outputs = x, name = 'inceptionv3')
    
    if use_BN_auxiliary:
        model = keras.models.Model(inputs = x_input, outputs = [x, logits], name = 'inceptionv3')
        
    return model

In [20]:
model = Inceptionv3()
print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
conv1_1_conv (Conv2D)           (None, 149, 149, 32) 864         input_5[0][0]                    
__________________________________________________________________________________________________
conv1_1_bn (BatchNormalization) (None, 149, 149, 32) 128         conv1_1_conv[0][0]               
__________________________________________________________________________________________________
conv1_1_relu (Activation)       (None, 149, 149, 32) 0           conv1_1_bn[0][0]                 
__________________________________________________________________________________________________
conv1_2_co

In [21]:
#生成框架图
#需要安装依赖环境
#S1. sudo yum install graphviz
#S2. sudo pip install graphviz
#S3. sudo pip install pydot
from keras.utils import plot_model
plot_model(model, to_file='model.png')

  下图是构建好的网络框架图:

  ![model](model.png)
  
  下面我们基于 keras.application 模块，快速搭建 inception v3 模型，并对下图进行类别预测：
  
  ![dog](dog.jpg)

In [12]:
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.applications.inception_v3 import preprocess_input, decode_predictions
import numpy as np

model = InceptionV3(weights='imagenet')
img_path = 'dog.jpg'
img = image.load_img(img_path, target_size=(299, 299))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

preds = model.predict(x)
print('Predicted:', decode_predictions(preds, top=3)[0])

96116736/96112376 [==============================] - 2543s 26us/step
Predicted: [('n02093428', 'American_Staffordshire_terrier', 0.76565653), ('n02093256', 'Staffordshire_bullterrier', 0.16094148), ('n02087394', 'Rhodesian_ridgeback', 0.0043874066)]


## 参考文献
(1)Szegedy C, Vanhoucke V, Ioffe S, et al. Rethinking the inception architecture for computer vision[C]//Proceedings of the IEEE conference on computer vision and pattern recognition. 2016: 2818-2826.  
(2)pytorch模型之Inception V3 https://zhuanlan.zhihu.com/p/30172532